# Example of data processing using sc2ts, tskit and VCZ

This notebook provides the code for the section of the paper in which we lay out the advantages of the software ecosystem that we are using.

In [1]:
import pathlib
import collections

import sc2ts
import tszip
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
datadir = pathlib.Path("../data")

In [3]:
ds = sc2ts.Dataset(datadir / "viridian_mafft_2024-10-14_v1.vcz.zip")
ds

Dataset at ../data/viridian_mafft_2024-10-14_v1.vcz.zip with 4484157 samples, 29903 variants, and 30 metadata fields. See ds.metadata.field_descriptors() for a description of the fields.

In [4]:
ts_sc2ts = tszip.load(datadir / "sc2ts_viridian_inter_v1.2.trees.tsz")

In [6]:
ts_usher = tszip.load(datadir / "usher_viridian_inter_v1.2.trees.tsz")

In [9]:
%%time 
dfn_sc2ts = sc2ts.node_data(ts_sc2ts)

CPU times: user 597 ms, sys: 59.8 ms, total: 657 ms
Wall time: 655 ms


In [8]:
%%time 
dfn_usher = sc2ts.node_data(ts_usher)

CPU times: user 937 ms, sys: 92.2 ms, total: 1.03 s
Wall time: 1.03 s


Make sure that the two tree sequences we're comparing have the same set of samples and reflects the same set of sequences.

In [10]:
assert np.array_equal(ts_sc2ts.samples(), ts_usher.samples())
assert np.array_equal(dfn_sc2ts[dfn_sc2ts.is_sample].node_id.values, ts_sc2ts.samples())
assert np.array_equal(dfn_usher[dfn_usher.is_sample].node_id.values, ts_usher.samples())
assert np.array_equal(dfn_usher[dfn_usher.is_sample].sample_id.values, dfn_sc2ts[dfn_sc2ts.is_sample].sample_id.values)
assert np.array_equal(ts_sc2ts.sites_position, ts_usher.sites_position)

A key element of processing data efficiently in tskit and VCZ is to use numpy arrays of integers to represent allelic states, instead of the classical approach of using strings, etc. In sc2ts, alleles are given fixed integer representations, such that A=0, C=1, G=2, and T=3. So, to represent the DNA string "AACTG" we would use the numpy array ``[0, 0, 1, 3, 2]`` instead. This has many advantages and makes it much easier to write efficient code. 

The drawback of this is that it's not as easy to inspect and debug, and we must always be aware of the translation required. 

In this analysis we're interested in how well the sc2ts and Usher do at imputing ambiguous bases, and want to count how many times the bases that they impute for samples are compatible with the ambiguity codes.

According to https://www.bioinformatics.org/sms/iupac.html the IUPAC ambiguity codes are as follows:

```
R	G or A	puRine
Y	T or C	pYrimidine
M	A or C	aMino
K	G or T	Keto
S	G or C	Strong interaction (3 H bonds)
W	A or T	Weak interaction (2 H bonds)
H	A or C or T	not-G, H follows G in the alphabet
B	G or T or C	not-A, B follows A
V	G or C or A	not-T (not-U), V follows U
D	G or A or T	not-C, D follows C
```

So, we build up a mapping of each ambiguity code to its compatible bases:

In [11]:
compatible = {
    "R": ["G", "A"],
    "Y": [ "T", "C"], 
   "M": ["A", "C"],
   "K": ["G", "T"],
   "S": ["G", "C"],
   "W": ["A", "T"],
   "H": ["A", "C", "T"],
   "B": ["G", "T", "C"],
   "V": ["G", "C", "A"],
   "D": ["G", "A", "T"],
}

The mapping from alleles to integers is managed by the sc2ts.IUPAC_ALLELES value, and so we build up the corresponding mapping in integer space:

In [12]:
sc2ts.IUPAC_ALLELES

'ACGT-RYSWKMBDHV.'

In [13]:
compatible_encoded = {}
for ambiguity_code, compatible_bases in compatible.items():
    compatible_bases_encoded = [sc2ts.IUPAC_ALLELES.index(base) for base in compatible_bases]
    compatible_encoded[sc2ts.IUPAC_ALLELES.index(ambiguity_code)] = compatible_bases_encoded
compatible_encoded

{5: [2, 0],
 6: [3, 1],
 10: [0, 1],
 9: [2, 3],
 7: [2, 1],
 8: [0, 3],
 13: [0, 1, 3],
 11: [2, 3, 1],
 14: [2, 1, 0],
 12: [2, 0, 3]}

A very important aspect of this data encoding is to understand how missing data is handled. In the tskit and VCF Zarr data encoding, the value -1 is reserved to mean "missing data" (for integer fields). Thus, while we can usually regard the integer endoded alleles as indexes into the list of alleles, we need to be careful with missing data as "-1" is treated as the "last element list" by Python. It is therefore important that we select out non-missing data *first* before we do any processing on the data that requires mapping back to the allelic state.

The following code block is the core of our analyis. We perform three parallel iterations over the sites in the SARS-CoV-2 genome that are shared by the Usher and sc2ts tree sequences, being careful to ensure that the same integer allele coding is used in all three. We extract the IDs  (strains) for the sample sequences from the node data table computed from the metadata by sc2ts above, and use this to extract the variants from the dataset in the correct sample order. We've already verified that the samples are in the same order in the Usher and sc2ts tree sequences above.

For each base we then extract the samples that have ambiguous base calls in the alignments, and then compute the number of these are imputed in a way that's consistent with the amibiguity code by Usher and sc2ts. We also compute the number of samples that are marked as Ns at each site, and the number of bases at which sc2ts and Usher differ.

This computation required 30 lines of code and took 5m58s on an intel i7-9700 CPU, with a peak memory usage of about 4.5G.

In [40]:
sample_id = dfn_usher[dfn_usher.is_sample].sample_id.values
ds_variants = ds.variants(sample_id=sample_id, position=ts_sc2ts.sites_position.astype(int))
usher_variants = ts_usher.variants(alleles=tuple(sc2ts.IUPAC_ALLELES))
sc2ts_variants = ts_sc2ts.variants(alleles=tuple(sc2ts.IUPAC_ALLELES))

iterator = tqdm(zip(ds_variants, usher_variants, sc2ts_variants), total=ts_usher.num_sites)
data = []
for var_ds, var_usher, var_sc2ts in iterator:
    usher_correctly_imputed = 0
    sc2ts_correctly_imputed = 0
    total_ambiguous = 0
    for ambiguity_code, compatible_bases in compatible_encoded.items():
        samples = np.where(var_ds.genotypes == ambiguity_code)[0]
        total_ambiguous += samples.shape[0]
        imputed = collections.Counter(var_usher.genotypes[samples])
        usher_correctly_imputed += sum(imputed[base] for base in compatible_bases)
        imputed = collections.Counter(var_sc2ts.genotypes[samples])
        sc2ts_correctly_imputed += sum(imputed[base] for base in compatible_bases)
    # IMPORTANT: -1 means missing data ("N")
    missing_samples = np.where(var_ds.genotypes == -1)[0]
    imputed_differently = np.sum(
        var_usher.genotypes[missing_samples] != var_sc2ts.genotypes[missing_samples])
       
    data.append({"position": int(var_ds.position), 
                 "total_ambiguous": total_ambiguous, 
                 "usher_correctly_imputed": usher_correctly_imputed,
                 "sc2ts_correctly_imputed": sc2ts_correctly_imputed,
                 "total_missing": missing_samples.shape[0],
                 "imputed_differently": imputed_differently
                })


  0%|          | 0/27507 [00:00<?, ?it/s]

In [15]:
df = pd.DataFrame(data)
df

,position,total_ambiguous,usher_correctly_imputed,sc2ts_correctly_imputed,total_missing,imputed_differently
0,267,1,1,1,21319,0
1,269,27,27,27,21247,0
2,270,12,12,12,21421,1
3,271,7,7,7,21415,0
4,272,16,16,16,21401,0
...,...,...,...,...,...,...
27502,29667,12,12,12,2044,0
27503,29668,43,43,43,1799,0
27504,29669,16,16,16,1801,0
27505,29670,55,55,55,1818,2


In [17]:
df["usher_incorrectly_imputed"] = df["total_ambiguous"] - df["usher_correctly_imputed"]
df["sc2ts_incorrectly_imputed"] = df["total_ambiguous"] - df["sc2ts_correctly_imputed"]
dfs = df.sum()
dfs

position                     409106633
total_ambiguous                 956859
usher_correctly_imputed         956852
sc2ts_correctly_imputed         956424
total_missing                162838461
imputed_differently             203579
usher_incorrectly_imputed            7
sc2ts_incorrectly_imputed          435
dtype: int64

In [36]:
print(f"{dfs.usher_incorrectly_imputed / dfs.total_ambiguous * 100: .4f}")

 0.0007


In [35]:
print(f"{dfs.sc2ts_incorrectly_imputed / dfs.total_ambiguous * 100: .2f}")

 0.05


In [33]:
print(f"{(dfs.imputed_differently / dfs.total_missing) * 100: .4f}")

 0.1250


In [20]:
(ts_sc2ts.num_samples * ts_sc2ts.num_sites) / 1024**3

63.41498617641628

In [30]:
ts_sc2ts.num_samples

2475418

In [32]:
ts_sc2ts.num_sites

27507

The total aligned dataset for 2,475,418 samples and 27,507 sites shared by both the sc2ts and Usher tree sequences represents 63.41GiB of nucleotide calls. Of the 162,838,461 missing data calls (Ns) in the alignments, sc2ts and Usher disagreed in their imputed values for 203,579 (0.125%). Additionally, 956,859 calls made use of the IUPAC uncertainty codes. Of these sc2ts imputed 435 (0.05%) incorrectly (i.e., with a base that is not compatible with the ambiguity code). Remarkably, Usher imputed only 7 calls (0.007%) from this set incorrectly.

## Tree operations

In [37]:
%%time
tree_usher = ts_usher.first()
num_polytomies = np.sum(tree_usher.num_children_array > 2)
largest_polytomy = np.max(tree_usher.num_children_array)
num_polytomies, largest_polytomy

CPU times: user 223 ms, sys: 4.08 ms, total: 227 ms
Wall time: 224 ms


(np.int64(261210), np.int32(15123))

In [38]:
%%time
num_polytomies = np.zeros(ts_sc2ts.num_trees, dtype=int)
largest_polytomy = np.zeros(ts_sc2ts.num_trees, dtype=int)
for tree in ts_sc2ts.trees():
    num_polytomies[tree.index] = np.sum(tree.num_children_array > 2)
    largest_polytomy[tree.index] = np.max(tree.num_children_array)
np.max(num_polytomies), np.max(largest_polytomy)

CPU times: user 765 ms, sys: 24 ms, total: 789 ms
Wall time: 787 ms


(np.int64(221487), np.int64(12663))

In [41]:
ts_sc2ts.num_trees

348

To illustrate the efficient access to phylogenetic analysis that tskit provides, we also computed the number of polytomies and the maximum number of a children per node. In the Usher tree it took 116ms and 4 lines of code to count polytomies (171,148) and compute the maximum number of children (5918). In the sc2ts ARG, it tool 385ms and 6 lines of code to find the maximum number of polytomies (137,001) and maximum number of children (7708) across all 348 local trees.